In [1]:
import tushare as ts
import pymongo
import datetime

In [2]:
client = pymongo.MongoClient("127.0.0.1", 27017)


In [5]:
def getHistoryData(code, startDate):
    res = ts.get_k_data(code, start=startDate)
    db = client.Trade
    collection = db[code]
    for k, v in res.iterrows():
        date = datetime.datetime.strptime(str(v.date), "%Y-%m-%d")
        item = {
            "date": date,
            "open": v.open,
            "close": v.close,
            "high": v.high,
            "low": v.low,
            "volume": v.volume
        }

        collection.replace_one({"date": date}, item, upsert=True)
    
    print("%s - %s rows" % (code, res.shape[0]))

In [6]:
collection = client.Common.Stock
today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
stocks = collection.find({})
for v in stocks:
    code = v["code"]
    if "kEndDate" in v:
        date = v["kEndDate"] + datetime.timedelta(days=-1)
    elif "foundTime" in v:
        date = v["foundTime"]
    else:
        date = datetime.datetime(2000, 1, 1)
    getHistoryData(code, date.strftime("%Y-%m-%d"))
    collection.update_one({"code": code}, {"$set": {"kEndDate": today}})

600363 - 0 rows
000099 - 0 rows
000096 - 0 rows
000090 - 0 rows
600783 - 0 rows
603648 - 1 rows
603139 - 1 rows
603138 - 0 rows
002193 - 0 rows
603136 - 1 rows
603131 - 0 rows
601368 - 0 rows
603133 - 0 rows
600780 - 0 rows
002196 - 0 rows
002194 - 0 rows
002195 - 0 rows
600958 - 0 rows
300252 - 0 rows
300253 - 0 rows
300250 - 0 rows
300251 - 0 rows
300256 - 0 rows
300257 - 0 rows
300254 - 0 rows
300255 - 0 rows
300258 - 0 rows
300259 - 0 rows
300568 - 0 rows
601360 - 0 rows
300494 - 0 rows
300495 - 0 rows
300496 - 0 rows
601866 - 0 rows
300490 - 0 rows
300491 - 0 rows
300492 - 0 rows
603730 - 1 rows
300498 - 0 rows
300499 - 0 rows
603738 - 0 rows
603229 - 1 rows
603228 - 0 rows
000625 - 0 rows
000622 - 0 rows
603223 - 0 rows
603222 - 0 rows
603225 - 0 rows
000971 - 0 rows
603227 - 0 rows
603226 - 0 rows
600369 - 0 rows
300427 - 0 rows
603919 - 0 rows
000520 - 0 rows
603917 - 1 rows
603916 - 1 rows
000153 - 0 rows
603912 - 1 rows
300617 - 0 rows
000525 - 0 rows
002815 - 0 rows
300684 -

002477 - 0 rows
002476 - 0 rows
002471 - 0 rows
002470 - 0 rows
002473 - 0 rows
002472 - 0 rows
002479 - 0 rows
002478 - 0 rows
600556 - 0 rows
600557 - 0 rows
600006 - 0 rows
600007 - 0 rows
600552 - 0 rows
000677 - 0 rows
600550 - 0 rows
600551 - 0 rows
001896 - 0 rows
601579 - 0 rows
600008 - 0 rows
600009 - 0 rows
600558 - 0 rows
600559 - 0 rows
600696 - 0 rows
000902 - 0 rows
600026 - 0 rows
603029 - 0 rows
603028 - 0 rows
603023 - 0 rows
603022 - 0 rows
603021 - 0 rows
603020 - 0 rows
000301 - 0 rows
603026 - 0 rows
603025 - 0 rows
300669 - 1 rows
300668 - 1 rows
300667 - 1 rows
300666 - 1 rows
300665 - 1 rows
300664 - 1 rows
300663 - 1 rows
300662 - 0 rows
300661 - 1 rows
300660 - 1 rows
000780 - 0 rows
000783 - 0 rows
000782 - 0 rows
000785 - 0 rows
000786 - 0 rows
000789 - 0 rows
000788 - 0 rows
603987 - 0 rows
601991 - 0 rows
601992 - 0 rows
601997 - 0 rows
601996 - 0 rows
601999 - 0 rows
601998 - 0 rows
600308 - 0 rows
603117 - 0 rows
603116 - 0 rows
603113 - 1 rows
603111 -

002697 - 1 rows
002696 - 0 rows
002691 - 0 rows
002690 - 0 rows
002693 - 0 rows
002692 - 0 rows
002459 - 0 rows
002458 - 0 rows
002457 - 0 rows
002456 - 0 rows
002455 - 0 rows
002454 - 0 rows
002453 - 0 rows
002452 - 0 rows
002451 - 0 rows
002450 - 0 rows
600570 - 0 rows
600571 - 0 rows
600572 - 0 rows
600573 - 0 rows
600575 - 0 rows
600576 - 0 rows
600577 - 0 rows
600578 - 0 rows
600579 - 0 rows
600206 - 0 rows
600207 - 0 rows
600202 - 0 rows
600203 - 0 rows
600200 - 0 rows
600201 - 0 rows
600208 - 0 rows
600209 - 0 rows
300688 - 1 rows
300609 - 0 rows
300608 - 0 rows
300601 - 0 rows
300600 - 0 rows
300603 - 0 rows
300602 - 0 rows
300605 - 0 rows
300604 - 1 rows
300607 - 0 rows
300606 - 1 rows
000755 - 0 rows
600371 - 0 rows
000752 - 0 rows
600757 - 0 rows
603167 - 0 rows
000506 - 0 rows
603165 - 0 rows
600262 - 0 rows
000617 - 0 rows
603177 - 0 rows
603179 - 1 rows
603178 - 0 rows
300735 - 1 rows
300737 - 1 rows
300736 - 1 rows
300731 - 1 rows
300730 - 1 rows
300733 - 1 rows
300732 -

000766 - 0 rows
603157 - 0 rows
000762 - 0 rows
000761 - 0 rows
000760 - 0 rows
300719 - 1 rows
300718 - 1 rows
300717 - 1 rows
300716 - 1 rows
300715 - 1 rows
300713 - 1 rows
300712 - 1 rows
300711 - 1 rows
300710 - 1 rows
600778 - 0 rows
600779 - 0 rows
300432 - 0 rows
300433 - 0 rows
300430 - 0 rows
601800 - 0 rows
300436 - 0 rows
300437 - 0 rows
300434 - 0 rows
300435 - 0 rows
300438 - 0 rows
300439 - 0 rows
603557 - 1 rows
603556 - 0 rows
603555 - 0 rows
603559 - 0 rows
000166 - 0 rows
603089 - 0 rows
603088 - 0 rows
601377 - 0 rows
603081 - 1 rows
603080 - 1 rows
603083 - 1 rows
603085 - 0 rows
000630 - 0 rows
300388 - 0 rows
300389 - 0 rows
300384 - 0 rows
300385 - 1 rows
300386 - 0 rows
300387 - 0 rows
300380 - 0 rows
300381 - 0 rows
300382 - 0 rows
300383 - 0 rows
603333 - 0 rows
603880 - 0 rows
603883 - 0 rows
000016 - 0 rows
603337 - 0 rows
000010 - 0 rows
300548 - 0 rows
603886 - 1 rows
603889 - 0 rows
603888 - 0 rows
603339 - 0 rows
300545 - 0 rows
300542 - 0 rows
000018 -

000701 - 0 rows
000700 - 0 rows
000703 - 0 rows
000702 - 0 rows
000705 - 0 rows
603896 - 1 rows
000707 - 0 rows
000498 - 0 rows
300390 - 0 rows
300696 - 1 rows
600099 - 0 rows
600887 - 0 rows
000635 - 0 rows
000637 - 0 rows
000636 - 0 rows
300418 - 0 rows
300419 - 0 rows
000633 - 0 rows
000632 - 0 rows
300414 - 0 rows
300415 - 0 rows
300416 - 0 rows
300417 - 0 rows
300410 - 0 rows
000638 - 0 rows
300412 - 0 rows
300413 - 0 rows
603533 - 1 rows
603535 - 1 rows
603536 - 1 rows
603538 - 0 rows
002282 - 0 rows
600302 - 0 rows
002280 - 0 rows
600090 - 0 rows
600097 - 0 rows
002287 - 0 rows
600095 - 0 rows
002285 - 0 rows
601919 - 0 rows
601918 - 0 rows
600048 - 0 rows
000039 - 0 rows
000038 - 0 rows
000037 - 0 rows
600882 - 0 rows
300562 - 0 rows
000034 - 0 rows
000032 - 0 rows
000031 - 0 rows
000030 - 0 rows
603628 - 0 rows
603199 - 0 rows
603198 - 0 rows
603197 - 0 rows
603196 - 1 rows
603626 - 0 rows
603843 - 0 rows
600793 - 0 rows
603848 - 1 rows
002746 - 0 rows
603997 - 0 rows
603996 -

000059 - 0 rows
603606 - 0 rows
603605 - 1 rows
603603 - 0 rows
603602 - 0 rows
603601 - 0 rows
603600 - 0 rows
000050 - 0 rows
000055 - 0 rows
603609 - 0 rows
603608 - 0 rows
603006 - 0 rows
300121 - 0 rows
603869 - 0 rows
600489 - 0 rows
603866 - 0 rows
603861 - 0 rows
603860 - 1 rows
603466 - 1 rows
300218 - 0 rows
300219 - 0 rows
300216 - 0 rows
300217 - 0 rows
300214 - 0 rows
300215 - 0 rows
300212 - 0 rows
300213 - 0 rows
300210 - 0 rows
300211 - 0 rows
603266 - 0 rows
603260 - 1 rows
603269 - 1 rows
603268 - 0 rows
000820 - 0 rows
300328 - 0 rows
300329 - 0 rows
300322 - 0 rows
300323 - 0 rows
300320 - 0 rows
300321 - 0 rows
300326 - 0 rows
300327 - 0 rows
300324 - 0 rows
300325 - 0 rows
300164 - 0 rows
002512 - 0 rows
600901 - 1 rows
601019 - 1 rows
300285 - 0 rows
601128 - 0 rows
600903 - 1 rows
603383 - 0 rows
601126 - 0 rows
601127 - 0 rows
000066 - 0 rows
300036 - 0 rows
300037 - 0 rows
300034 - 0 rows
300035 - 0 rows
300032 - 0 rows
300033 - 0 rows
300030 - 0 rows
300031 -

002401 - 0 rows
603937 - 1 rows
603936 - 0 rows
002150 - 0 rows
603933 - 1 rows
002151 - 0 rows
603939 - 0 rows
603938 - 1 rows
300304 - 0 rows
300305 - 0 rows
300306 - 0 rows
300307 - 0 rows
300300 - 0 rows
300301 - 0 rows
300302 - 0 rows
300303 - 0 rows
300308 - 0 rows
300309 - 0 rows
600727 - 0 rows
600381 - 0 rows
000955 - 0 rows
600380 - 0 rows
300683 - 1 rows
002912 - 1 rows
002913 - 1 rows
002910 - 1 rows
002911 - 1 rows
002916 - 1 rows
002917 - 1 rows
002915 - 1 rows
002918 - 1 rows
002919 - 1 rows
600919 - 0 rows
300010 - 0 rows
300011 - 0 rows
300012 - 0 rows
300013 - 0 rows
300014 - 0 rows
300015 - 0 rows
300016 - 0 rows
300017 - 0 rows
300018 - 0 rows
300019 - 0 rows
002358 - 0 rows
002359 - 0 rows
600547 - 0 rows
002350 - 0 rows
002351 - 0 rows
002352 - 0 rows
002353 - 0 rows
002354 - 0 rows
002355 - 0 rows
002356 - 0 rows
002357 - 0 rows
600708 - 0 rows
600879 - 0 rows
600875 - 0 rows
600702 - 0 rows
600877 - 0 rows
600876 - 0 rows
600871 - 0 rows
600706 - 0 rows
600873 -